In [36]:
import time
start = time.time()
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import glob
print(time.time()-start,'s')

0.00045609474182128906 s


In [98]:
def order_points(pts,crop,main=0):
    ''' The function returns the 4 corner points in a set of points. 
    The input are a set of points (pts).
    crop is the amount of pixels that are subtracted from the four points obtained which is kind of padding.
    main is the flag which exclusively runs when the ordering points are of the main form'''
    rect = np.zeros((4, 2), dtype = "float32")
 

    s = pts.sum(axis = 1)
#     print(s)
#     print(pts)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
 

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    if main:
        rect[1][1]-=110
        rect[0][0] -=20
    
    #cropping a little more towards left
    rect[0][0] -=crop
    rect[3][0] -=crop
    #cropping a little more towards right
    rect[1][0] +=crop
    rect[2][0] +=crop
    #cropping a little more towards bottom
    rect[3][1] +=crop
    rect[2][1] +=crop
 
    return rect

In [99]:
def get_outer_points(pts,crop):

    rect = np.zeros((4, 2), dtype = "float32")
 
    min_x = min(pts[:,0])
    min_y = min(pts[:,1])
    max_x = max(pts[:,0])
    max_y = max(pts[:,1])
    #s = pts.sum(axis = 1)
    #print(s)
    rect[0][0] = min_x
    rect[0][1] = min_y
    rect[2][0] = max_x
    rect[2][1] = max_y
 

    #diff = np.diff(pts, axis = 1)
    #print(diff)
    rect[1][0] = max_x
    rect[1][1] = min_y
    rect[3][0] = min_x
    rect[3][1] = max_y
    
    #cropping a little more towards left
    rect[0][0] -=crop
    rect[3][0] -=crop
    #cropping a little more towards right
    rect[1][0] +=crop
    rect[2][0] +=crop
    #cropping a little more towards bottom
    rect[3][1] +=crop
    rect[2][1] +=crop
 
    return rect

In [100]:
def four_point_warp(image, pts,crop,outer=1,main=0):

    if outer:
        rect = get_outer_points(pts,crop)
    else :
        rect = order_points(pts,crop,main)
    (tl, tr, br, bl) = rect
    #print(rect)

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
              [0, 0],
              [maxWidth - 1, 0],
              [maxWidth - 1, maxHeight - 1],
              [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    cv2.imwrite('Out.jpg',warped)
 
    return warped

In [101]:
def compare_features_orb(gray_main, gray_crop, mask_tl=None, mask_br=None):
    #Trying feature matching using ORB
    orb = cv2.ORB_create()
    Mask = None
    #cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)
    #cv2.rectangle(mask,(int(width/10),int(2*height/17)),(int(13*width/21),int(4*height/29)),(255),-1)
    #cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)
    if mask_tl and mask_br:
        Mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
        cv2.rectangle(Mask, mask_tl, mask_br, (255), thickness = -1)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(gray_main,mask)
    kp2, des2 = orb.detectAndCompute(gray_crop,None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(des1,des2)

    matches = sorted(matches, key = lambda x:x.distance)

    img = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,matches,None, flags=2)

    #cv2.imwrite('Out1.jpg',img)
    return np.float32([ kp1[m.queryIdx].pt for m in matches ])

In [102]:
def compare_features(gray_main, gray_crop, mask_tl=None, mask_br=None, Mask_main=None):
    #Trying feature matching using SIFT
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    Mask=Mask_main
    #mask for residential address
    #cv2.rectangle(mask, (int(0),int(18*height/25)),(width,int(25*height/26)), (255), thickness = -1)
    
    if mask_tl and mask_br:
        if Mask_main is not None:
            cv2.rectangle(Mask, mask_tl, mask_br, (0), thickness = -1)
        else :
            Mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
            cv2.rectangle(Mask, mask_tl, mask_br, (255), thickness = -1)
    
    #mask for branch
    #cv2.rectangle(mask,(int(width/10),int(3*height/27)),(int(10*width/11),int(3*height/20)),(255),-1)
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(gray_main,Mask)
    kp2, des2 = sift.detectAndCompute(gray_crop,None)
    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    matches = flann.knnMatch(des1,des2,k=2,mask=Mask)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.8*n.distance:
            good.append(m)
    
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    
    h,w = gray_main.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    
    
    img2 = cv2.polylines(gray_crop,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

    img3 = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,good,None,**draw_params)
    cv2.imwrite('Out1.jpg',img3)
    return np.float32([ kp1[m.queryIdx].pt for m in good ])

In [103]:
def read_input(main_file, crop_file):
    img1 = cv2.imread(main_file)
    crop = cv2.imread(crop_file)
    gray_main = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
    height_main, width_main = gray_main.shape
    height_crop, width_crop = gray_crop.shape
    height = height_main
    width = width_main
    #print(height_main, width_main)
    #print(height_crop, width_crop)
    return img1, gray_main, gray_crop, height, width
    #cv2.imwrite('Out.jpg',gray_main)

In [104]:
def crop_selected_rectangle(img, tl, br):
    return img[tl[1]:br[1],tl[0]:br[0]]

In [97]:
start_time = time.time()

types = ('*.jpg', '*.JPG','*.JPEG', '*.jpeg')
files = []
os.system("mkdir crop")
os.chdir(os.getcwd() + "/form-samples/")
for t in types:
    if glob.glob(t) != [] :
        files.append(glob.glob(t))
os.chdir("../crop/")
i=1
for f in files[0]:
    print(f)
    img1, gray_main, gray_crop, height, width = read_input('../form-samples/'+f, '../form_print.jpg')
    mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)+255
    points = compare_features(gray_main, gray_crop, (0,int(height/4)),(width,int(18*height/25)),mask)
    crop = 30
    dst = four_point_warp(gray_main, points, crop, outer=0, main=1)
    height, width = dst.shape
    
    os.system("mkdir crop"+str(i))
    branch = crop_selected_rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)))
    print('Saving into crop'+str(i))
    os.chdir(os.getcwd()+'/crop'+str(i))
    cv2.imwrite('branch.jpg',branch)
    os.chdir('../')
    print('Time taken : ',time.time()-start_time)
    i=i+1
    #print(height,width)


filled-pa.jpg
3507 2481
3504 2479
[[  264.26904297   443.30407715]
 [ 2294.56762695   449.793396  ]
 [ 2271.60058594  3355.45654297]
 [  208.64698792  3335.47485352]]
Saving into crop1
Time taken :  88.55214929580688
filled-s1.jpg
3507 2481
3504 2479
[[  404.8944397    282.56170654]
 [ 2412.24902344   523.42895508]
 [ 2063.37841797  3405.13061523]
 [    9.07249069  3152.79516602]]
Saving into crop2
Time taken :  159.86106705665588
filled-s2.jpg
3507 2481
3504 2479
[[   20.54397583   396.99411011]
 [ 2054.51904297   254.73965454]
 [ 2251.46191406  3151.02172852]
 [  193.62646484  3291.29003906]]
Saving into crop3
Time taken :  236.05063772201538


In [123]:
crop = 30
dst = four_point_warp(gray_main, points, crop, outer=0, main=1)
print(time.time()-start_time)
height, width = dst.shape
print(height,width)

1191.1624717712402
2902 2062


In [124]:
# Crop branch
# cv2.rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)),(0),3)
# branch = crop_selected_rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)))
cv2.rectangle(dst,(int(20*width/30),int(height/20)),(int(34*width/35),int(3*height/37)),(0),3)
cv2.imwrite('Out.jpg',dst)

True

In [96]:
i=0
# os.system("mkdir crop")
#os.chdir('..')
# os.system("mkdir crop"+str(i))
print(os.getcwd())
str('a')

/home/darshan/CS/Summer2018/Impel/OCR/Darshan/Photos


'a'

In [73]:
# For residential address
# address_x = (int(0),int(18*height/25))
# address_y = (width,int(25*height/26))

# points = compare_features(gray_main, gray_crop, address_x, address_y)
# points = points[points[:,1]>(7*height)/10]